ここに追加していく

In [1]:
!pip install requests beautifulsoup4


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse

In [4]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse

START_URL = "https://www.musashino-u.ac.jp/"

# 大学のドメイン（これ以外のリンクは辿らないようにするため）
BASE_DOMAIN = urlparse(START_URL).netloc 

# 課題の指示通りの辞書
# key: URL, value: <title>内の文字列
page_titles = {}

# これから訪問すべきURLのリスト（キュー）
urls_to_visit = [START_URL]

# 既に訪問したURLのセット（重複と無限ループを防ぐため）
visited_urls = set()
visited_urls.add(START_URL)

print(f"--- クローリング開始: {START_URL} ---")

# urls_to_visit リストが空になるまでループ
#（※処理が多すぎると時間がかかりすぎるため、50ページで一旦停止する制限をかけます）
limit = 50 

while urls_to_visit and len(page_titles) < limit:
    # 1. リストの先頭からURLを1つ取り出す
    current_url = urls_to_visit.pop(0)

    print(f"[{len(page_titles)+1} / {limit}] Processing: {current_url}") # 進捗確認用

    # 2. 負荷軽減のためにスリープ (重要！)
    time.sleep(1) # 1秒待つ

    try:
        # 3. Webページにアクセス
        response = requests.get(current_url, timeout=5) # 5秒でタイムアウト
        
        # 4. HTMLの文字コードを（可能なら）ヘッダーから判定
        response.encoding = response.apparent_encoding
        
        # エラーならスキップ
        if response.status_code != 200:
            print(f"  -> Error: Status Code {response.status_code}")
            continue

        # 5. HTMLを解析
        soup = BeautifulSoup(response.text, 'html.parser')

        # 6. <title>を取得し、辞書に格納
        title = "No Title Found" # デフォルト値
        if soup.title and soup.title.string:
            title = soup.title.string.strip() # .strip()で前後の空白を削除
        
        page_titles[current_url] = title

        # 7. ページ内の全<a>タグ（リンク）を取得
        for link in soup.find_all('a'):
            href = link.get('href')

            # 8. リンクを絶対URLに変換
            if not href:
                continue # href属性がないタグは無視
            
            absolute_url = urljoin(current_url, href)
            
            # URLの#以降（ページ内リンク）や?以降（クエリ）を削除
            absolute_url = absolute_url.split('#')[0]
            absolute_url = absolute_url.split('?')[0]
            
            # .pdfや.jpgなどのファイルへのリンクは無視（任意）
            if any(ext in absolute_url.lower() for ext in ['.pdf', '.jpg', '.png', '.zip', '.xlsx', '.doc', '.ppt']):
                continue

            # 9. リンクをチェック
            # (a) 同じドメインか？
            # (b) まだ訪問済みリストに入っていないか？
            if urlparse(absolute_url).netloc == BASE_DOMAIN and absolute_url not in visited_urls:
                urls_to_visit.append(absolute_url) # 訪問リストに追加
                visited_urls.add(absolute_url)     # 訪問済みリストに追加

    except requests.exceptions.RequestException as e:
        print(f"  -> Error accessing {current_url}: {e}")
        continue # エラーが起きても処理を続ける

# 10. 最後に結果をprint
print("\n--- クローリング完了 ---")
if len(page_titles) >= limit:
    print(f"（{limit}ページの上限に達しました）")

print("【取得結果（辞書型変数）】")
print(page_titles)

--- クローリング開始: https://www.musashino-u.ac.jp/ ---
[1 / 50] Processing: https://www.musashino-u.ac.jp/
[2 / 50] Processing: https://www.musashino-u.ac.jp/access.html
[3 / 50] Processing: https://www.musashino-u.ac.jp/admission/request.html
[4 / 50] Processing: https://www.musashino-u.ac.jp/contact.html
[5 / 50] Processing: https://www.musashino-u.ac.jp/prospective-students.html
[6 / 50] Processing: https://www.musashino-u.ac.jp/students.html
[7 / 50] Processing: https://www.musashino-u.ac.jp/alumni.html
[8 / 50] Processing: https://www.musashino-u.ac.jp/parents.html
[9 / 50] Processing: https://www.musashino-u.ac.jp/business.html
[10 / 50] Processing: https://www.musashino-u.ac.jp/guide/
[11 / 50] Processing: https://www.musashino-u.ac.jp/guide/profile/
[12 / 50] Processing: https://www.musashino-u.ac.jp/guide/activities/
[13 / 50] Processing: https://www.musashino-u.ac.jp/guide/campus/
[14 / 50] Processing: https://www.musashino-u.ac.jp/guide/facility/
[15 / 50] Processing: https://www.